In [ ]:
# 기존 설치 명령을 다음으로 교체
%pip install --upgrade transformers torch sentence-transformers pandas tqdm# 첫 번째 셀에서 실행
# 기존 패키지 완전 제거 후 재설치
%pip uninstall -y transformers sentence-transformers torch torchvision torchaudio


ERROR: Invalid requirement: 'tqdm#': Expected end or semicolon (after name and no valid version specifier)
    tqdm#
        ^
Note: you may need to restart the kernel to use updated packages.
Found existing installation: transformers 4.54.1
Uninstalling transformers-4.54.1:
  Successfully uninstalled transformers-4.54.1
Found existing installation: sentence-transformers 5.0.0
Uninstalling sentence-transformers-5.0.0:
  Successfully uninstalled sentence-transformers-5.0.0
Found existing installation: torch 2.7.1
Uninstalling torch-2.7.1:
  Successfully uninstalled torch-2.7.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 사전 컴파일된 바이너리 패키지로 안전하게 설치
%pip install --no-build-isolation --only-binary=all torch torchvision torchaudio
%pip install --no-build-isolation --only-binary=all transformers==4.21.3
%pip install --no-build-isolation --only-binary=all sentence-transformers==2.2.2
%pip install pandas numpy tqdm scikit-learn

  Using cached torch-2.7.1-cp313-none-macosx_11_0_arm64.whl.metadata (29 kB)
Using cached torch-2.7.1-cp313-none-macosx_11_0_arm64.whl (68.6 MB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  Preparing metadata (pyproject.toml) ...   Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "/opt/anaconda3/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "/opt/anaconda3/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook

In [ ]:
# MPS 디바이스 오류 해결을 위한 환경 설정
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

# 패키지 버전 확인 및 재설치
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cpu --force-reinstall --no-deps
!pip install transformers==4.44.2 --force-reinstall --no-deps
!pip install sentence-transformers==3.0.1 --force-reinstall --no-deps
!pip install tokenizers==0.19.1 --force-reinstall --no-deps

In [ ]:
# 라이브러리 임포트 (오류 처리 강화)
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# PyTorch를 CPU 전용으로 설정
import torch
torch.set_default_device('cpu')
device = torch.device('cpu')
print(f"사용 중인 디바이스: {device}")

# Sentence Transformers를 CPU 전용으로 import
try:
    from sentence_transformers import SentenceTransformer, util
    print("Sentence Transformers 로드 성공")
    SBERT_AVAILABLE = True
except Exception as e:
    print(f"Sentence Transformers 로드 실패: {e}")
    SBERT_AVAILABLE = False

# TF-IDF를 위한 sklearn import
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import gc
import psutil

print("기본 라이브러리 로드 완료")

ModuleNotFoundError: Could not import module 'AutoModelForSequenceClassification'. Are this object's requirements defined correctly?

In [ ]:
# 디바이스 설정 및 모델 로드
if torch.backends.mps.is_available():
    device = 'mps'
    print("🍎 Mac GPU (MPS)")
elif torch.cuda.is_available():
    device = 'cuda'
    print("🚀 NVIDIA GPU (CUDA)")
else:
    device = 'cpu'
    print("💻 CPU")

# 모델 로드 (오류 처리 강화)
model = None

if USE_SBERT and torch:
    try:
        print("🔄 KoSentenceBERT 모델 로드 시도...")
        model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
        
        # 디바이스 이동 시도
        try:
            model = model.to(device)
            print(f"✅ 모델을 {device}로 이동완료")
        except Exception as e:
            print(f"⚠️ {device} 이동 실패: {e}")
            device = 'cpu'
            model = model.to(device)
            print("🔄 CPU로 fallback")
            
        print(f"✅ KoSentenceBERT 모델 로드 성공")
        print(f"🔧 최종 사용 디바이스: {device}")
        
    except Exception as e:
        print(f"❌ SBERT 모델 로드 실패: {e}")
        print("🔄 TF-IDF 기반 대안 시스템으로 전환")
        USE_SBERT = False
        model = None

if not USE_SBERT:
    # TF-IDF 기반 대안 시스템
    print("🔧 TF-IDF 기반 유사도 계산 시스템 준비")
    
    def calculate_similarity_tfidf(originals, summaries):
        """TF-IDF를 사용한 유사도 계산"""
        print("📊 TF-IDF 벡터화 및 유사도 계산 중...")
        
        # TF-IDF 벡터라이저 설정
        vectorizer = TfidfVectorizer(
            max_features=10000,
            ngram_range=(1, 3),
            min_df=2,
            max_df=0.95,
            strip_accents='unicode',
            lowercase=True
        )
        
        # 모든 텍스트 합쳐서 벡터화
        all_texts = originals + summaries
        tfidf_matrix = vectorizer.fit_transform(all_texts)
        
        # 원본과 요약 분리
        n_originals = len(originals)
        original_vectors = tfidf_matrix[:n_originals]
        summary_vectors = tfidf_matrix[n_originals:]
        
        # 각 쌍별 코사인 유사도 계산
        similarities = []
        for i in tqdm(range(len(originals)), desc="유사도 계산"):
            sim = cosine_similarity(
                original_vectors[i:i+1], 
                summary_vectors[i:i+1]
            )[0][0]
            similarities.append(sim)
        
        return np.array(similarities)
    
    print("✅ TF-IDF 대안 시스템 준비 완료")

print(f"\\n🎯 최종 설정:")
print(f"- 평가 방법: {'KoSentenceBERT' if USE_SBERT else 'TF-IDF 코사인 유사도'}")
print(f"- 디바이스: {device if USE_SBERT and torch else 'CPU'}")
print(f"- 모델 상태: {'로드됨' if model else '대안 시스템 사용'}")

In [ ]:
# 데이터 파일 불러오기
import os
import pandas as pd

# 현재 디렉토리 확인
current_dir = os.getcwd()
print(f"현재 작업 디렉토리: {current_dir}")

# 파일 존재 확인
files_to_check = ["crawling_origin.csv", "news_summaries.csv"]
available_files = []

for file in files_to_check:
    if os.path.exists(file):
        available_files.append(file)
        print(f"✅ {file} 파일 존재")
    else:
        print(f"❌ {file} 파일 없음")

# 사용 가능한 CSV 파일 자동 검색
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
print(f"\n📁 현재 디렉토리의 CSV 파일들: {csv_files}")

try:
    # 원본 파일 로드 시도
    if "crawling_origin.csv" in available_files:
        original_df = pd.read_csv("crawling_origin.csv")
        print(f"✅ crawling_origin.csv 로드 성공: {original_df.shape}")
    else:
        # 대체 파일 찾기
        origin_candidates = [f for f in csv_files if any(keyword in f.lower() 
                           for keyword in ['origin', 'original', 'crawl', '원본'])]
        if origin_candidates:
            original_df = pd.read_csv(origin_candidates[0])
            print(f"✅ 대체 원본 파일 사용: {origin_candidates[0]} - {original_df.shape}")
        else:
            raise FileNotFoundError("원본 데이터 파일을 찾을 수 없습니다")
    
    # 요약 파일 로드 시도
    if "news_summaries.csv" in available_files:
        summary_df = pd.read_csv("news_summaries.csv")
        print(f"✅ news_summaries.csv 로드 성공: {summary_df.shape}")
    else:
        # 대체 파일 찾기
        summary_candidates = [f for f in csv_files if any(keyword in f.lower() 
                            for keyword in ['summary', 'summarized', '요약', 'kobert'])]
        if summary_candidates:
            summary_df = pd.read_csv(summary_candidates[0])
            print(f"✅ 대체 요약 파일 사용: {summary_candidates[0]} - {summary_df.shape}")
        else:
            raise FileNotFoundError("요약 데이터 파일을 찾을 수 없습니다")
    
    print(f"\n📊 데이터 로드 완료")
    print(f"원본 전문: {original_df.shape}")
    print(f"KoBERT 요약: {summary_df.shape}")
    
    # 데이터 컬럼 확인
    print(f"\n📋 컬럼 정보:")
    print(f"원본 컬럼: {original_df.columns.tolist()}")
    print(f"요약 컬럼: {summary_df.columns.tolist()}")
    
    # 데이터 미리보기
    print(f"\n👀 원본 데이터 미리보기:")
    print(original_df.head(2))
    print(f"\n👀 요약 데이터 미리보기:")
    print(summary_df.head(2))
    
except FileNotFoundError as e:
    print(f"❌ 파일 오류: {e}")
    print(f"💡 해결 방법:")
    print(f"1. 파일명을 확인하세요: {files_to_check}")
    print(f"2. 현재 디렉토리에 파일이 있는지 확인하세요")
    print(f"3. 사용 가능한 CSV 파일: {csv_files}")
except Exception as e:
    print(f"❌ 데이터 로드 오류: {e}")
    print(f"파일 인코딩 문제일 수 있습니다. UTF-8로 저장되었는지 확인하세요.")

In [ ]:
# 데이터 병합 및 컬럼 매핑
try:
    # 데이터프레임이 제대로 로드되었는지 확인
    if 'original_df' not in locals() or 'summary_df' not in locals():
        raise ValueError("데이터가 로드되지 않았습니다. 이전 셀을 먼저 실행하세요.")
    
    print(f"🔍 데이터 병합 준비:")
    print(f"원본 데이터: {original_df.shape}")
    print(f"요약 데이터: {summary_df.shape}")
    
    # 길이가 같으면 인덱스 기준으로 병합
    if len(original_df) == len(summary_df):
        data = pd.concat([original_df.reset_index(drop=True), summary_df.reset_index(drop=True)], axis=1)
        print("✅ 인덱스 기준 병합 완료")
    else:
        # 길이가 다르면 최소 길이로 맞춤
        min_len = min(len(original_df), len(summary_df))
        data = pd.concat([
            original_df.head(min_len).reset_index(drop=True), 
            summary_df.head(min_len).reset_index(drop=True)
        ], axis=1)
        print(f"⚠️ 길이 불일치로 {min_len}개로 조정하여 병합")
    
    # 중복 컬럼명 처리
    data.columns = pd.io.common.dedup_names(data.columns, is_potential_multiindex=False)
    
    # 원본 전문 컬럼 자동 감지 (더 넓은 범위로)
    original_cols = []
    for col in original_df.columns:
        col_lower = str(col).lower()
        if any(keyword in col_lower for keyword in [
            'content', 'article', 'text', '본문', '전문', 'body', 
            'news', '뉴스', 'story', '기사', 'description'
        ]):
            original_cols.append(col)
    
    # 요약문 컬럼 자동 감지 (더 넓은 범위로)
    summary_cols = []
    for col in summary_df.columns:
        col_lower = str(col).lower()
        if any(keyword in col_lower for keyword in [
            'summary', '요약', 'kobert', 'summarized', 'abstract',
            '줄거리', '개요', 'brief'
        ]):
            summary_cols.append(col)
    
    print(f"\\n🔎 컬럼 감지 결과:")
    print(f"원본 후보 컬럼: {original_cols}")
    print(f"요약 후보 컬럼: {summary_cols}")
    
    # 컬럼 선택
    if original_cols and summary_cols:
        original_col = original_cols[0]
        summary_col = summary_cols[0]
        print(f"✅ 자동 감지 성공 - 원본: '{original_col}', 요약: '{summary_col}'")
    else:
        # 자동 감지 실패시 수동 설정 가이드
        print("❌ 컬럼 자동 감지 실패. 수동 설정이 필요합니다.")
        print("\\n📋 사용 가능한 컬럼들:")
        print("원본 데이터:", original_df.columns.tolist())
        print("요약 데이터:", summary_df.columns.tolist())
        
        # 사용자가 직접 지정할 수 있도록 가이드
        print("\\n💡 아래 코드를 수정하여 올바른 컬럼명을 지정하세요:")
        print("original_col = 'YOUR_ORIGINAL_COLUMN_NAME'")
        print("summary_col = 'YOUR_SUMMARY_COLUMN_NAME'")
        
        # 기본값으로 첫 번째 컬럼 사용
        if len(original_df.columns) > 0 and len(summary_df.columns) > 0:
            original_col = original_df.columns[0]
            summary_col = summary_df.columns[0]
            print(f"\\n⚠️ 임시로 첫 번째 컬럼 사용 - 원본: '{original_col}', 요약: '{summary_col}'")
        else:
            raise ValueError("사용할 수 있는 컬럼이 없습니다.")
    
    # 텍스트 데이터 추출 및 검증
    if original_col in data.columns and summary_col in data.columns:
        # NaN 값을 빈 문자열로 변환 후 문자열로 변환
        originals = data[original_col].fillna('').astype(str).tolist()
        summaries = data[summary_col].fillna('').astype(str).tolist()
        
        print(f"\\n📊 데이터 추출 완료:")
        print(f"병합된 데이터: {data.shape}")
        print(f"원본 텍스트: {len(originals)}개")
        print(f"요약 텍스트: {len(summaries)}개")
        
        # 데이터 품질 체크
        empty_originals = sum(1 for x in originals if not str(x).strip() or str(x).strip() in ['nan', 'None'])
        empty_summaries = sum(1 for x in summaries if not str(x).strip() or str(x).strip() in ['nan', 'None'])
        
        print(f"\\n🔍 데이터 품질:")
        print(f"빈 원본: {empty_originals}개")
        print(f"빈 요약: {empty_summaries}개")
        
        # 샘플 텍스트 확인
        if len(originals) > 0 and len(summaries) > 0:
            # 유효한 샘플 찾기
            valid_sample_idx = 0
            for i, (orig, summ) in enumerate(zip(originals, summaries)):
                if (str(orig).strip() and str(summ).strip() and 
                    str(orig).strip() not in ['nan', 'None'] and 
                    str(summ).strip() not in ['nan', 'None']):
                    valid_sample_idx = i
                    break
            
            print(f"\\n📝 샘플 확인 (인덱스 {valid_sample_idx}):")
            print(f"원본 예시 ({len(originals[valid_sample_idx])}자): {originals[valid_sample_idx][:200]}...")
            print(f"요약 예시 ({len(summaries[valid_sample_idx])}자): {summaries[valid_sample_idx][:150]}...")
    else:
        available_cols = list(data.columns)
        raise ValueError(f"지정된 컬럼을 찾을 수 없습니다. 사용 가능한 컬럼: {available_cols}")
        
except Exception as e:
    print(f"❌ 데이터 처리 오류: {e}")
    print(f"\\n💡 해결 방법:")
    print(f"1. 이전 셀이 성공적으로 실행되었는지 확인")
    print(f"2. 데이터 파일의 컬럼명을 확인")
    print(f"3. 데이터 파일의 인코딩을 UTF-8로 저장")
    if 'data' in locals():
        print(f"4. 현재 사용 가능한 컬럼: {list(data.columns)}")

In [ ]:
# 샘플링 및 데이터 전처리
try:
    # 이전 단계 검증
    if 'originals' not in locals() or 'summaries' not in locals():
        raise ValueError("텍스트 데이터가 준비되지 않았습니다. 이전 셀을 먼저 실행하세요.")
    
    USE_SAMPLE = True
    SAMPLE_SIZE = 500  # KoBERT 요약 평가용 샘플 크기

    print(f"🔍 데이터 전처리 시작...")
    print(f"전체 데이터: {len(originals)}개")

    # 유효한 데이터만 필터링 (더 엄격한 조건)
    valid_pairs = []
    for i, (orig, summ) in enumerate(zip(originals, summaries)):
        # 문자열 변환 및 정리
        orig_str = str(orig).strip()
        summ_str = str(summ).strip()
        
        # 유효성 검사
        if (orig_str and summ_str and 
            orig_str not in ['nan', '', 'None', 'null'] and 
            summ_str not in ['nan', '', 'None', 'null'] and
            len(orig_str) > 50 and  # 원본은 최소 50자
            len(summ_str) > 10 and  # 요약은 최소 10자
            len(orig_str) < 10000 and  # 원본은 최대 10000자 (너무 긴 텍스트 제외)
            len(summ_str) < 2000):   # 요약은 최대 2000자
            valid_pairs.append((i, orig_str, summ_str))

    print(f"🔍 데이터 품질 확인:")
    print(f"전체 데이터: {len(originals)}개")
    print(f"유효한 데이터: {len(valid_pairs)}개")
    print(f"제거된 데이터: {len(originals) - len(valid_pairs)}개")
    print(f"유효 비율: {len(valid_pairs)/len(originals)*100:.1f}%")

    if len(valid_pairs) == 0:
        print("❌ 유효한 데이터가 없습니다.")
        print("💡 문제 해결:")
        print("1. 데이터 파일의 내용을 확인하세요")
        print("2. 텍스트 컬럼이 올바른지 확인하세요")
        print("3. 빈 값이나 NaN 값이 너무 많은지 확인하세요")
        raise ValueError("유효한 데이터가 없습니다.")

    # 샘플링 적용
    if USE_SAMPLE and len(valid_pairs) > SAMPLE_SIZE:
        np.random.seed(42)  # 재현 가능한 결과를 위해
        sample_indices = np.random.choice(len(valid_pairs), SAMPLE_SIZE, replace=False)
        selected_pairs = [valid_pairs[i] for i in sample_indices]
        print(f"🚀 샘플 모드: {len(selected_pairs)}개 데이터로 평가")
    else:
        selected_pairs = valid_pairs
        print(f"📊 전체 데이터 모드: {len(selected_pairs)}개 데이터로 평가")

    # 최종 데이터 준비
    if len(selected_pairs) > 0:
        indices, originals_clean, summaries_clean = zip(*selected_pairs)
        originals_clean = list(originals_clean)
        summaries_clean = list(summaries_clean)
        
        # 통계 계산
        orig_lengths = [len(text) for text in originals_clean]
        summ_lengths = [len(text) for text in summaries_clean]
        compression_ratios = [len(summ)/len(orig)*100 for orig, summ in zip(originals_clean, summaries_clean)]
        
        print(f"\\n✅ 최종 평가 데이터: {len(originals_clean)}개")
        print(f"📏 원본 길이 - 평균: {np.mean(orig_lengths):.0f}자, 중간값: {np.median(orig_lengths):.0f}자")
        print(f"📏 요약 길이 - 평균: {np.mean(summ_lengths):.0f}자, 중간값: {np.median(summ_lengths):.0f}자")
        print(f"📊 압축 비율 - 평균: {np.mean(compression_ratios):.1f}%, 중간값: {np.median(compression_ratios):.1f}%")
        
        # 품질 샘플 확인
        print(f"\\n📝 처리된 데이터 샘플:")
        sample_idx = len(originals_clean) // 2  # 중간 위치 샘플
        print(f"원본: {originals_clean[sample_idx][:150]}...")
        print(f"요약: {summaries_clean[sample_idx][:100]}...")
        
    else:
        raise ValueError("선택된 데이터가 없습니다.")
        
except Exception as e:
    print(f"❌ 데이터 전처리 오류: {e}")
    print(f"\\n💡 해결 방법:")
    print(f"1. 이전 셀들이 모두 성공적으로 실행되었는지 확인")
    print(f"2. SAMPLE_SIZE를 더 작게 설정 (예: 100)")
    print(f"3. 데이터 파일의 품질을 확인")
    
    # 디버깅 정보 제공
    if 'originals' in locals() and 'summaries' in locals():
        print(f"\\n🔍 디버깅 정보:")
        print(f"originals 타입: {type(originals)}, 길이: {len(originals)}")
        print(f"summaries 타입: {type(summaries)}, 길이: {len(summaries)}")
        if len(originals) > 0:
            print(f"첫 번째 원본 예시: {str(originals[0])[:100]}...")
        if len(summaries) > 0:
            print(f"첫 번째 요약 예시: {str(summaries[0])[:100]}...")

In [ ]:
# 통합 평가 함수 (SBERT + TF-IDF 지원)
def get_memory_info():
    """현재 디바이스의 메모리 상태 반환"""
    try:
        if USE_SBERT and torch:
            if device == 'mps':
                return "🍎 MPS 사용 중"
            elif device == 'cuda':
                try:
                    allocated = torch.cuda.memory_allocated() / 1024**3
                    cached = torch.cuda.memory_reserved() / 1024**3
                    return f"🚀 CUDA - 할당: {allocated:.2f}GB, 캐시: {cached:.2f}GB"
                except:
                    return "🚀 CUDA 메모리 정보 불가"
            else:
                return "💻 CPU 모드"
        else:
            return "💻 TF-IDF 모드 (CPU)"
    except Exception as e:
        return f"❌ 메모리 정보 오류: {e}"

def encode_with_progress(model, texts, batch_size=16, description="Encoding"):
    """안전한 배치 임베딩 처리"""
    if not texts:
        raise ValueError("텍스트 리스트가 비어있습니다.")
    
    embeddings = []
    failed_batches = 0
    print(f"시작: {get_memory_info()}")
    print(f"처리할 텍스트: {len(texts)}개, 배치 크기: {batch_size}")
    
    try:
        for i in tqdm(range(0, len(texts), batch_size), desc=description):
            batch_texts = texts[i:i+batch_size]
            
            # 빈 텍스트 제거 및 길이 제한
            cleaned_batch = []
            for text in batch_texts:
                text_str = str(text).strip()
                if text_str and len(text_str) > 5:  # 최소 5자 이상
                    # 너무 긴 텍스트는 잘라내기 (메모리 보호)
                    if len(text_str) > 5000:
                        text_str = text_str[:5000]
                    cleaned_batch.append(text_str)
            
            if not cleaned_batch:
                print(f"⚠️ 배치 {i//batch_size}: 유효한 텍스트가 없어 건너뜀")
                failed_batches += 1
                continue
                
            try:
                # 임베딩 생성
                batch_embeddings = model.encode(
                    cleaned_batch, 
                    convert_to_tensor=True, 
                    show_progress_bar=False,
                    device=device,
                    batch_size=min(len(cleaned_batch), 8)  # 내부 배치 크기 제한
                )
                embeddings.append(batch_embeddings)
                
                # 메모리 정리 (5배치마다)
                if i % (batch_size * 5) == 0:
                    try:
                        if device == 'cuda':
                            torch.cuda.empty_cache()
                        elif device == 'mps':
                            torch.mps.empty_cache()
                    except Exception as cache_error:
                        print(f"⚠️ 캐시 정리 오류: {cache_error}")
                        
            except Exception as batch_error:
                print(f"⚠️ 배치 {i//batch_size} 처리 중 오류: {batch_error}")
                failed_batches += 1
                
                # 배치 크기를 줄여서 재시도
                if len(cleaned_batch) > 1:
                    try:
                        print(f"🔄 배치 크기를 1로 줄여서 재시도...")
                        for single_text in cleaned_batch:
                            single_emb = model.encode(
                                [single_text], 
                                convert_to_tensor=True, 
                                show_progress_bar=False,
                                device=device
                            )
                            embeddings.append(single_emb)
                    except Exception as retry_error:
                        print(f"❌ 재시도도 실패: {retry_error}")
                        continue
        
        if not embeddings:
            raise ValueError("모든 배치 처리에 실패했습니다.")
        
        # 결과 합치기
        result = torch.cat(embeddings, dim=0)
        success_rate = (len(texts) - failed_batches * batch_size) / len(texts) * 100
        
        print(f"완료: {get_memory_info()}")
        print(f"📊 처리 결과: {result.shape[0]}개 임베딩 생성 (성공률: {success_rate:.1f}%)")
        
        if failed_batches > 0:
            print(f"⚠️ 실패한 배치: {failed_batches}개")
        
        return result
        
    except Exception as e:
        print(f"❌ 임베딩 생성 중 치명적 오류: {e}")
        
        # 메모리 정리 시도
        try:
            if device == 'cuda':
                torch.cuda.empty_cache()
            elif device == 'mps':
                torch.mps.empty_cache()
        except:
            pass
            
        raise e

def evaluate_similarity(originals_clean, summaries_clean):
    """통합 유사도 평가 함수"""
    print(f"🔄 유사도 평가 시작...")
    print(f"📊 평가 데이터: {len(originals_clean)}개")
    print(f"🎯 평가 방법: {'KoSentenceBERT' if USE_SBERT else 'TF-IDF'}")
    print(f"💾 시작 메모리: {get_memory_info()}")
    
    if USE_SBERT and model:
        # SBERT 기반 평가
        try:
            print("\\n1️⃣ 원본 전문 임베딩 생성...")
            
            # 배치별 임베딩 처리
            def encode_batch(texts, batch_size=8):
                embeddings = []
                for i in tqdm(range(0, len(texts), batch_size), desc="임베딩"):
                    batch_texts = texts[i:i+batch_size]
                    try:
                        batch_emb = model.encode(
                            batch_texts,
                            convert_to_tensor=True,
                            show_progress_bar=False,
                            device=device,
                            batch_size=min(4, len(batch_texts))
                        )
                        embeddings.append(batch_emb)
                        
                        # 메모리 정리
                        if torch and i % 40 == 0:
                            if device == 'cuda':
                                torch.cuda.empty_cache()
                            elif device == 'mps':
                                torch.mps.empty_cache()
                    except Exception as e:
                        print(f"⚠️ 배치 {i//batch_size} 오류: {e}")
                        continue
                
                return torch.cat(embeddings, dim=0) if embeddings else None
            
            original_emb = encode_batch(originals_clean)
            if original_emb is None:
                raise ValueError("원본 임베딩 생성 실패")
                
            print("\\n2️⃣ 요약문 임베딩 생성...")
            summary_emb = encode_batch(summaries_clean)
            if summary_emb is None:
                raise ValueError("요약 임베딩 생성 실패")
            
            print("\\n3️⃣ 코사인 유사도 계산...")
            similarities = util.cos_sim(summary_emb, original_emb).diagonal()
            similarity_scores = similarities.cpu().numpy()
            
            print(f"✅ SBERT 평가 완료")
            
        except Exception as e:
            print(f"❌ SBERT 평가 실패: {e}")
            print("🔄 TF-IDF로 대체 평가")
            similarity_scores = calculate_similarity_tfidf(originals_clean, summaries_clean)
    else:
        # TF-IDF 기반 평가
        similarity_scores = calculate_similarity_tfidf(originals_clean, summaries_clean)
    
    print(f"\\n💾 완료 메모리: {get_memory_info()}")
    return similarity_scores

# 함수 준비 확인
try:
    if 'originals_clean' not in locals() or 'summaries_clean' not in locals():
        raise ValueError("정제된 데이터가 준비되지 않았습니다. 이전 셀을 먼저 실행하세요.")
    
    print(f"🎯 평가 시스템 준비 완료")
    print(f"📊 평가 대상: {len(originals_clean)}개 문서")
    print(f"🔧 평가 방법: {'KoSentenceBERT' if USE_SBERT and model else 'TF-IDF'}")
    print(f"💾 현재 메모리: {get_memory_info()}")
    
    # 데이터 최종 검증
    if len(originals_clean) != len(summaries_clean):
        raise ValueError(f"원본과 요약 개수가 다릅니다: {len(originals_clean)} vs {len(summaries_clean)}")
    
    print(f"✅ 모든 준비 완료")
    
except Exception as e:
    print(f"❌ 준비 단계 오류: {e}")
    print(f"💡 이전 셀들을 순서대로 다시 실행해주세요.")

In [ ]:
# KoBERT 요약 성능 평가 실행
print("🔄 KoBERT 요약 성능 평가 시작...")
print(f"시작점: {get_memory_info()}")

try:
    # 1. 원본 전문 임베딩
    print("\n1️⃣ 원본 전문 임베딩 생성...")
    original_emb = encode_with_progress(
        model, 
        originals_clean, 
        batch_size=16,  # 메모리 안정성을 위해 작은 배치
        description="원본 전문"
    )
    
    # 2. KoBERT 요약 임베딩
    print("\n2️⃣ KoBERT 요약 임베딩 생성...")
    summary_emb = encode_with_progress(
        model, 
        summaries_clean, 
        batch_size=16,
        description="KoBERT 요약"
    )
    
    # 3. 의미 유사도 계산
    print("\n3️⃣ 의미 유사도 계산...")
    similarities = util.cos_sim(summary_emb, original_emb).diagonal()
    
    # 결과 통계 계산
    score_mean = similarities.mean().item()
    score_max = similarities.max().item()
    score_min = similarities.min().item()
    score_std = similarities.std().item()
    
    print("\n" + "="*50)
    print("🎯 KoBERT 요약 성능 평가 결과")
    print("="*50)
    print(f"📊 평균 의미 유사도: {score_mean:.4f}")
    print(f"📈 최고 유사도: {score_max:.4f}")
    print(f"📉 최저 유사도: {score_min:.4f}")
    print(f"📏 표준편차: {score_std:.4f}")
    
    # 성능 등급 분류
    if score_mean >= 0.80:
        grade = "🏆 우수 (Excellent)"
        comment = "매우 높은 품질의 요약"
    elif score_mean >= 0.70:
        grade = "👍 양호 (Good)"
        comment = "양질의 요약 성능"
    elif score_mean >= 0.60:
        grade = "⚠️ 보통 (Average)"
        comment = "개선 여지가 있는 요약"
    elif score_mean >= 0.50:
        grade = "📝 미흡 (Below Average)"
        comment = "상당한 개선 필요"
    else:
        grade = "❌ 부족 (Poor)"
        comment = "요약 품질 재검토 필요"
    
    print(f"\n🎖️ 종합 평가: {grade}")
    print(f"💬 평가 의견: {comment}")
    
    # 분포 분석
    similarity_scores = similarities.cpu().numpy()
    high_quality = np.sum(similarity_scores >= 0.8)
    good_quality = np.sum((similarity_scores >= 0.7) & (similarity_scores < 0.8))
    avg_quality = np.sum((similarity_scores >= 0.6) & (similarity_scores < 0.7))
    low_quality = np.sum(similarity_scores < 0.6)
    
    print(f"\n📈 품질 분포:")
    print(f"🏆 우수 (≥0.8): {high_quality}개 ({high_quality/len(similarity_scores)*100:.1f}%)")
    print(f"👍 양호 (0.7-0.8): {good_quality}개 ({good_quality/len(similarity_scores)*100:.1f}%)")
    print(f"⚠️ 보통 (0.6-0.7): {avg_quality}개 ({avg_quality/len(similarity_scores)*100:.1f}%)")
    print(f"📝 개선필요 (<0.6): {low_quality}개 ({low_quality/len(similarity_scores)*100:.1f}%)")
    
except Exception as e:
    print(f"❌ 평가 중 오류 발생: {e}")
    print("💡 해결 방법:")
    print("- 배치 크기를 더 줄여보세요 (batch_size=8)")
    print("- 샘플 크기를 줄여보세요 (SAMPLE_SIZE=100)")
    print("- VS Code를 재시작해보세요")

In [ ]:
# 우수 요약 및 개선 필요 요약 분석
if 'similarity_scores' in locals():
    # 상위 3개 우수 요약
    top_indices = np.argsort(similarity_scores)[-3:]
    print("\n" + "="*60)
    print("🏆 우수한 요약 사례 (상위 3개)")
    print("="*60)
    
    for i, idx in enumerate(top_indices[::-1]):
        print(f"\n🥇 {i+1}위 - 유사도: {similarity_scores[idx]:.4f}")
        print("─" * 40)
        print(f"📄 원본 ({len(originals_clean[idx])}자): {originals_clean[idx][:200]}...")
        print(f"📝 요약 ({len(summaries_clean[idx])}자): {summaries_clean[idx][:150]}...")
        print(f"📊 압축률: {len(summaries_clean[idx])/len(originals_clean[idx])*100:.1f}%")
    
    # 하위 3개 개선 필요 요약
    bottom_indices = np.argsort(similarity_scores)[:3]
    print("\n" + "="*60)
    print("📝 개선 필요 요약 사례 (하위 3개)")
    print("="*60)
    
    for i, idx in enumerate(bottom_indices):
        print(f"\n⚠️ {i+1}위 - 유사도: {similarity_scores[idx]:.4f}")
        print("─" * 40)
        print(f"📄 원본 ({len(originals_clean[idx])}자): {originals_clean[idx][:200]}...")
        print(f"📝 요약 ({len(summaries_clean[idx])}자): {summaries_clean[idx][:150]}...")
        print(f"📊 압축률: {len(summaries_clean[idx])/len(originals_clean[idx])*100:.1f}%")
    
    print(f"\n🔧 최종 메모리 상태: {get_memory_info()}")
    print("\n✅ KoBERT 요약 성능 평가 완료!")
else:
    print("❌ 평가 결과가 없습니다. 위의 셀을 먼저 실행하세요.")

In [ ]:
# 결과 저장 (선택사항)
if 'similarity_scores' in locals() and 'selected_pairs' in locals():
    # 결과 데이터프레임 생성
    results_df = pd.DataFrame({
        'index': [pair[0] for pair in selected_pairs],
        'original_text': originals_clean,
        'summary_text': summaries_clean,
        'similarity_score': similarity_scores,
        'original_length': [len(text) for text in originals_clean],
        'summary_length': [len(text) for text in summaries_clean],
        'compression_ratio': [len(summ)/len(orig)*100 for orig, summ in zip(originals_clean, summaries_clean)]
    })
    
    # 성능 등급 추가
    def get_grade(score):
        if score >= 0.8: return "우수"
        elif score >= 0.7: return "양호"
        elif score >= 0.6: return "보통"
        else: return "개선필요"
    
    results_df['quality_grade'] = results_df['similarity_score'].apply(get_grade)
    
    # 결과 저장
    output_filename = "kobert_evaluation_results.csv"
    results_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    
    print(f"💾 평가 결과가 '{output_filename}'에 저장되었습니다.")
    print(f"📊 저장된 데이터: {len(results_df)}행 × {len(results_df.columns)}열")
    
    # 요약 통계 출력
    print(f"\n📈 저장된 결과 요약:")
    print(results_df[['similarity_score', 'original_length', 'summary_length', 'compression_ratio']].describe())
    
    print(f"\n🏷️ 품질 등급별 분포:")
    print(results_df['quality_grade'].value_counts())
    
else:
    print("❌ 저장할 평가 결과가 없습니다.")